In [ ]:
# pip install pymysql

In [ ]:
# pip install mariadb

In [65]:
import pandas as pd
import copy

In [66]:
from mariadb import MariaDB
with open('./aws_config','r',encoding='utf-8') as f:
    db_config = {lines.split('=')[0].strip():lines.split('=')[1].strip() for lines in f.readlines()}
db = MariaDB(db_config)

In [67]:
# rating_dart = pd.read_excel('rating_dart_total.xlsx')
# dart = rating_dart[['회사명', '년도', '총점수']].copy()
# dart.rename(columns={'총점수':'다트점수'}, inplace=True)
# dart.set_index(['회사명', '년도'], inplace=True)
# dart

In [68]:
rating_patent = pd.read_csv('특허_점수.csv', sep='\t')
patent = rating_patent[['회사명', '년도', '점수']].copy()
patent.rename(columns={'점수':'특허점수'}, inplace=True)
patent.set_index(['회사명', '년도'], inplace=True)
patent

특허점수
회사명     년도        
CJ ENM  2016   1.0
        2020   1.0
        2022   1.0
CJ프레시웨이 2014   1.0
        2015   1.0
...            ...
휴온스글로벌  2012   1.0
        2014   1.0
        2015   1.0
        2021   1.0
        2023   1.0

[2605 rows x 1 columns]

In [69]:
rating_green = pd.read_csv('녹색점수.csv')
green = rating_green[['기업명', '인증연도', '점수']].copy()
green.columns = ['회사명', '년도', '녹색점수']
green.set_index(['회사명', '년도'], inplace=True)
green

녹색점수
회사명   년도        
국일제지  2021     2
뉴보텍   2024     2
대원산업  2024     2
대창스틸  2023     2
      2024     2
...          ...
하츠    2024     2
한라IMS 2024     2
한탑    2024     2
해성산업  2021     2
효성오앤비 2022     2

[63 rows x 1 columns]

In [70]:
rating_recall = pd.read_csv('리콜점수.csv')
recall = rating_recall[['기업명', '인증연도', '점수']].copy()
recall.columns = ['회사명', '년도', '리콜점수']
recall.set_index(['회사명', '년도'], inplace=True)
recall

리콜점수
회사명     년도        
더네이쳐홀딩스 2019     4
동일금속    2021     4
로보로보    2015     4
머큐리     2014     3
바른손     2015     4
배럴      2019     4
비엠티     2018     4
삼천리자전거  2019     4
        2022     2
        2023     4
        2023     4
서울리거    2016     4
스튜디오삼익  2021     4
        2023     4
알톤스포츠   2022     4
        2023     4
        2023     4
에코아이    2015     4
        2017     4
        2023     4
오로라     2021     4
정다운     2016     2
        2017     3
제로투세븐   2015     4
제이에스티나  2019     4
토박스코리아  2017     4
        2019     3
하츠      2024     4

In [71]:
rating_bert = pd.read_csv('t_kosdaq_exec_df.csv')
bert = rating_bert[['기업', '년도', 'E', 'S', 'G', 'P', 'N']].copy()
bert['E*P'] = bert['E'] * bert['P'] * 8
bert['S*P'] = bert['S'] * bert['P'] * 20
bert['G*P'] = bert['G'] * bert['P'] * 15

# # '년도'가 2022 또는 2023인 경우 다른 계산식 적용
# import numpy as np
# bert['G*P'] = np.where(
#     (bert['년도'] == 2022) | (bert['년도'] == 2023),
#     bert['G'] * bert['P'] * 20,  # 다트 있는 경우 (2023, 2022)
#     bert['G'] * bert['P'] * 25   # 다트 없는 경우
# )

bert = bert[['기업', '년도', 'E*P', 'S*P', 'G*P', 'P', 'N']].copy()
bert.columns = ['회사명', '년도', 'bert_e', 'bert_s', 'bert_g', 'Company_Pos', 'Company_Neg']
bert.set_index(['회사명', '년도'], inplace=True)
bert

bert_e    bert_s    bert_g  Company_Pos  Company_Neg
회사명        년도                                                          
오상헬스케어     2019  1.967508  5.702600  4.391986       0.5289       0.4711
           2020  2.117860  6.087304  4.774793       0.5719       0.4281
           2021  2.267218  6.390324  5.120523       0.6121       0.3879
           2022  2.184249  6.231950  4.919867       0.5897       0.4103
           2023  2.008548  5.734852  4.463859       0.5397       0.4603
...                   ...       ...       ...          ...          ...
하나32호스팩    2020  2.774912  7.609563  6.644869       0.7605       0.2395
           2019  2.502975  7.128604  5.722144       0.6806       0.3194
IBKS제24호스팩 2024  2.457009  6.781482  5.647028       0.6642       0.3358
BF랩스       2024  2.408966  6.769836  5.493155       0.6522       0.3478
SG&G       2024  2.235481  6.560494  5.123684       0.6051       0.3949

[9824 rows x 5 columns]

In [72]:
dff = pd.concat([green, recall, bert, patent],axis=0)
# dff = pd.concat([dart, green, recall, bert],axis=0)

# 회사명을 기준으로 정렬
dff.sort_values(by=['회사명', '년도'], inplace=True)
dff = dff.groupby(['회사명', '년도']).sum()
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수  
회사명 년도          
3S  2019   0.0  
    2020   0.0  
    2021   0.0  
    2022   0.0  
    2023   0.0  
...        ...  
힘스  2020   0.0  
    2021   0.0  
    2022   0.0  
    2023   0.0  
    2024   0.0  

[11379 rows x 8 columns]

In [73]:
rating_job = pd.read_csv('kosdaq_jobplanet_numer_data_result.csv')
job = rating_job[['회사명', 's_point', 'g_point']].copy()
job.columns = ['회사명', 'job_s', 'job_g']
job

,회사명,job_s,job_g
0,아이엠비디엑스,1.703667,2.130000
1,엔젤로보틱스,1.778000,1.742000
2,제이투케이바이오,4.633333,8.666667
3,삼현,2.138500,1.831333
4,오상헬스케어,1.200000,1.157333
...,...,...,...
1714,한국전자인증,3.963750,1.973333
1715,BGF에코머티리얼즈,0.000000,0.000000
1716,NPX,0.000000,0.000000
1717,하나32호스팩,0.000000,0.000000


In [74]:
# job 데이터를 MultiIndex 데이터프레임에 채워 넣기
for idx, row in job.iterrows():
    company = row['회사명']
    if company in dff.index.get_level_values(0):
        for year in dff.loc[company].index:
            dff.loc[(company, year), 'job_s'] = row['job_s']
            dff.loc[(company, year), 'job_g'] = row['job_g']
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s  job_g  
회사명 년도                           
3S  2019   0.0  0.986000  3.538  
    2020   0.0  0.986000  3.538  
    2021   0.0  0.986000  3.538  
    2022   0.0  0.986000  3.538  
    2023   0.0  0.986000  3.538  
...        ...       ...    ...  
힘스  2020   0.0  1.085083  2.592  
    2021   0.0  1.085083  2.592  
    2022   0.0  1.085083  2.592  
    2023   0.0  1.085083  2.592  
    2024   0.0  1.085083  2.592  

[11379 rows x 10 columns]

In [75]:
# # 결측치인 행의 인덱스 반환
# dff['job_s'][dff['job_s'].isnull()]

In [76]:
# # 결측치는 0으로 채우기
# dff['job_s'] = dff['job_s'].fillna(0)
# dff['job_g'] = dff['job_g'].fillna(0)
# dff

In [77]:
# '년도' 인덱스를 검사하여 2023, 2022가 아닌 경우 'job_g' 값을 1.5배로 조정
# dff['job_g'] = dff.apply(lambda x: x['job_g'] * 1.5 if x.name[1] not in [2023, 2022] else x['job_g'], axis=1)
dff['job_g'] *= 3
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g  
회사명 년도                            
3S  2019   0.0  0.986000  10.614  
    2020   0.0  0.986000  10.614  
    2021   0.0  0.986000  10.614  
    2022   0.0  0.986000  10.614  
    2023   0.0  0.986000  10.614  
...        ...       ...     ...  
힘스  2020   0.0  1.085083   7.776  
    2021   0.0  1.085083   7.776  
    2022   0.0  1.085083   7.776  
    2023   0.0  1.085083   7.776  
    2024   0.0  1.085083   7.776  

[11379 rows x 10 columns]

In [78]:
# 점수 합계 구하기
dff['E점수(100)'] = ((dff['bert_e'] + dff['녹색점수'] + dff['특허점수'])/15)*100
dff['S점수(100)'] = ((dff['bert_s'] + dff['job_s'] + dff['리콜점수'])/40)*100
# dff['G점수(100)'] = ((dff['bert_g'] + dff['job_g'] + dff['다트점수'])/45)*100
dff['G점수(100)'] = ((dff['bert_g'] + dff['job_g'])/45)*100
# dff['총점수(100)'] = dff['다트점수']  + dff['녹색점수'] + dff['리콜점수'] + dff['bert_e'] + dff['bert_s'] + dff['bert_g'] + dff['job_s'] + dff['job_g'] + dff['특허점수']
dff['총점수(100)'] = dff['녹색점수'] + dff['리콜점수'] + dff['bert_e'] + dff['bert_s'] + dff['bert_g'] + dff['job_s'] + dff['job_g'] + dff['특허점수']
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g   E점수(100)   S점수(100)   G점수(100)   총점수(100)  
회사명 년도                                                                        
3S  2019   0.0  0.986000  10.614  14.468826  17.943107  34.441013  24.846023  
    2020   0.0  0.986000  10.614  14.589293  18.091751  34.543185  24.969527  
    2021   0.0  0.986000  10.614  14.758912  18.150320  34.631995  25.058362  
    2022   0.0  0.986000  10.614  15.201003  18.625480  35.015253  25.487206  
    2023   0.0  0.986000  10.614  15.743188  19.077887  35.425005  25.933885  
...        ...       ...     ...        ...        ...        ...        ...  
힘스  2020   0.0  1.085083   7.776  12.760202  16.481524  26.802744  20.567875  
    2021   0.0  1.085083   7.776  14.142528  17.889888  27.841597  21.806053  
    2022   0.0  1.085083   7.776  14.613312  18.225401  28.228980  22.185198  
    2023   0.0  1.085083   7.776  13.559127  17.132744  27.291791  21.168273  
    2024   0.0  1.085083   7.776  14.398129  18.078507  28.054123  22.015478  

[11379 rows x 14 columns]

In [79]:
# 회사명과 년도 기준으로 데이터프레임 정렬
dff.sort_values(by=['회사명', '년도'], inplace=True)

# 각 점수에 대한 이전 년도와의 차이 계산
for column in ['E점수(100)', 'S점수(100)', 'G점수(100)', '총점수(100)']:
    dff[f'{column} 차이'] = dff.groupby('회사명')[column].diff()
dff

녹색점수  리콜점수    bert_e    bert_s    bert_g  Company_Pos  Company_Neg  \
회사명 년도                                                                         
3S  2019   0.0   0.0  2.170324  6.191243  4.884456       0.5843       0.4157   
    2020   0.0   0.0  2.188394  6.250700  4.930433       0.5898       0.4102   
    2021   0.0   0.0  2.213837  6.274128  4.970398       0.5964       0.4036   
    2022   0.0   0.0  2.280150  6.464192  5.142864       0.6140       0.3860   
    2023   0.0   0.0  2.361478  6.645155  5.327252       0.6359       0.3641   
...        ...   ...       ...       ...       ...          ...          ...   
힘스  2020   0.0   0.0  1.914030  5.507526  4.285235       0.5153       0.4847   
    2021   0.0   0.0  2.121379  6.070872  4.752718       0.5710       0.4290   
    2022   0.0   0.0  2.191997  6.205077  4.927041       0.5895       0.4106   
    2023   0.0   0.0  2.033869  5.768014  4.505306       0.5458       0.4542   
    2024   0.0   0.0  2.159719  6.146320  4.848355       0.5827       0.4173   

          특허점수     job_s   job_g   E점수(100)   S점수(100)   G점수(100)   총점수(100)  \
회사명 년도                                                                         
3S  2019   0.0  0.986000  10.614  14.468826  17.943107  34.441013  24.846023   
    2020   0.0  0.986000  10.614  14.589293  18.091751  34.543185  24.969527   
    2021   0.0  0.986000  10.614  14.758912  18.150320  34.631995  25.058362   
    2022   0.0  0.986000  10.614  15.201003  18.625480  35.015253  25.487206   
    2023   0.0  0.986000  10.614  15.743188  19.077887  35.425005  25.933885   
...        ...       ...     ...        ...        ...        ...        ...   
힘스  2020   0.0  1.085083   7.776  12.760202  16.481524  26.802744  20.567875   
    2021   0.0  1.085083   7.776  14.142528  17.889888  27.841597  21.806053   
    2022   0.0  1.085083   7.776  14.613312  18.225401  28.228980  22.185198   
    2023   0.0  1.085083   7.776  13.559127  17.132744  27.291791  21.168273   
    2024   0.0  1.085083   7.776  14.398129  18.078507  28.054123  22.015478   

          E점수(100) 차이  S점수(100) 차이  G점수(100) 차이  총점수(100) 차이  
회사명 년도                                                        
3S  2019          NaN          NaN          NaN          NaN  
    2020     0.120467     0.148644     0.102172     0.123505  
    2021     0.169619     0.058569     0.088810     0.088835  
    2022     0.442091     0.475160     0.383259     0.428844  
    2023     0.542186     0.452407     0.409752     0.446679  
...               ...          ...          ...          ...  
힘스  2020     0.715609     1.064022     0.694795     0.845608  
    2021     1.382326     1.408364     1.038853     1.238178  
    2022     0.470784     0.335513     0.387383     0.379145  
    2023    -1.054185    -1.092657    -0.937189    -1.016925  
    2024     0.839001     0.945763     0.762332     0.847205  

[11379 rows x 18 columns]

In [80]:
dff = dff.reset_index()
dff

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [81]:
sorted(list(dff['총점수(100)'].unique()))[-1]

53.90366432

In [82]:
# # 특정 행 추출
# dff[(dff['회사명'] == '심텍홀딩스')]

In [83]:
# # 열 순서 변경
# df = df[['회사명', '년도', 'bert_e', '녹색점수', '특허점수', 'E점수(100)', 'E점수(100) 차이', 'bert_s', 'job_s', '리콜점수', 'S점수(100)', 'S점수(100) 차이', 'bert_g', 'job_g', '다트점수', 'G점수(100)', 'G점수(100) 차이', '총점수(100)', '총점수(100) 차이']]
# df

In [84]:
dff

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [135]:
normal_df = dff.copy()
normal_df

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,0.0,0.0,2.170324,6.191243,4.884456,0.5843,0.4157,0.0,0.986000,10.614,14.468826,17.943107,34.441013,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,0.0,0.0,2.188394,6.250700,4.930433,0.5898,0.4102,0.0,0.986000,10.614,14.589293,18.091751,34.543185,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,0.0,0.0,2.213837,6.274128,4.970398,0.5964,0.4036,0.0,0.986000,10.614,14.758912,18.150320,34.631995,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,0.0,0.0,2.280150,6.464192,5.142864,0.6140,0.3860,0.0,0.986000,10.614,15.201003,18.625480,35.015253,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,0.0,0.0,2.361478,6.645155,5.327252,0.6359,0.3641,0.0,0.986000,10.614,15.743188,19.077887,35.425005,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,0.0,0.0,1.914030,5.507526,4.285235,0.5153,0.4847,0.0,1.085083,7.776,12.760202,16.481524,26.802744,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,0.0,0.0,2.121379,6.070872,4.752718,0.5710,0.4290,0.0,1.085083,7.776,14.142528,17.889888,27.841597,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,0.0,0.0,2.191997,6.205077,4.927041,0.5895,0.4106,0.0,1.085083,7.776,14.613312,18.225401,28.228980,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,0.0,0.0,2.033869,5.768014,4.505306,0.5458,0.4542,0.0,1.085083,7.776,13.559127,17.132744,27.291791,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [136]:
def min_max_normalization(data, min_val, max_val):
    normalized_data = []
    for val in data:
        normalized_val = (val - min(data)) / (max(data) - min(data)) * (max_val - min_val) + min_val
        normalized_data.append(normalized_val)
    return normalized_data

data = [15, 0.23, 0.45, 0.67, 0.89, 1.0, 1.5, 2.0, 5.0, 10.0, 0.2]  # 데이터가 숫자로 채워져야 합니다.
min_val = 3
max_val = 15

normalized_data = min_max_normalization(data, min_val, max_val)
print(normalized_data)

[15.0, 3.0243243243243243, 3.2027027027027026, 3.381081081081081, 3.5594594594594593, 3.6486486486486487, 4.054054054054054, 4.45945945945946, 6.891891891891891, 10.945945945945946, 3.0]


In [137]:
bert_escore = [score for score in dff['bert_e']]
bert_sscore = [score for score in dff['bert_s']]
bert_gscore = [score for score in dff['bert_g']]
job_sscore = [score for score in dff['job_s']]
job_gscore = [score for score in dff['job_g']]
patent_score = [score for score in dff['특허점수']]
green_score = [score for score in dff['녹색점수']]
recall_score = [score for score in dff['리콜점수']]

normal_df['bert_e'] = min_max_normalization(bert_escore, 15,8/15*100 ) # max 값 : 53
normal_df['녹색점수'] = min_max_normalization(green_score,10,3.5/15*100 ) # 23
normal_df['특허점수'] = min_max_normalization(patent_score,13,3.5/15*100 ) # 23

normal_df['bert_s'] = min_max_normalization(bert_sscore,30 ,50 )
normal_df['리콜점수'] = min_max_normalization(recall_score,15,25 )
normal_df['job_s'] = min_max_normalization(job_sscore,10,25 )

normal_df['job_g'] = min_max_normalization(job_gscore,20,50 )
normal_df['bert_g'] = min_max_normalization(bert_gscore,30,50 )

In [138]:
### E,S,G 점수 구하기

normal_df['E점수(100)'] = normal_df['bert_e'] + normal_df['녹색점수'] + normal_df['특허점수']
normal_df['S점수(100)'] = normal_df['bert_s'] + normal_df['job_s'] + normal_df['리콜점수']
normal_df['G점수(100)'] = normal_df['bert_g'] + normal_df['job_g']

In [139]:
normal_df

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,13.0,15.0,35.895478,39.676778,39.596042,0.5843,0.4157,13.0,11.875000,30.614,61.895478,66.551778,70.210042,24.846023,NaN,NaN,NaN,NaN
1,3S,2020,13.0,15.0,36.027823,39.769709,39.686369,0.5898,0.4102,13.0,11.875000,30.614,62.027823,66.644709,70.300369,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,13.0,15.0,36.214167,39.806326,39.764884,0.5964,0.4036,13.0,11.875000,30.614,62.214167,66.681326,70.378884,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,13.0,15.0,36.699849,40.103392,40.103712,0.6140,0.3860,13.0,11.875000,30.614,62.699849,66.978392,70.717712,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,13.0,15.0,37.295495,40.386233,40.465963,0.6359,0.3641,13.0,11.875000,30.614,63.295495,67.261233,71.079963,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,13.0,15.0,34.018381,38.608144,38.418807,0.5153,0.4847,13.0,12.063419,27.776,60.018381,65.671563,66.194807,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,13.0,15.0,35.537006,39.488641,39.337229,0.5710,0.4290,13.0,12.063419,27.776,61.537006,66.552060,67.113229,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,13.0,15.0,36.054211,39.698401,39.679705,0.5895,0.4106,13.0,12.063419,27.776,62.054211,66.761820,67.455705,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,13.0,15.0,34.896082,39.015281,38.851161,0.5458,0.4542,13.0,12.063419,27.776,60.896082,66.078700,66.627161,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


최솟값, 최댓값, 평균값
85.0 94.0160101024021 88.68040300337121
78.03623647734956 90.67602978621838 83.01493469224731
70.416 98.29281250471506 78.24896254585785


In [129]:
for i, e in enumerate(normal_df['총점수(100)']) :
    if e == min(normal_df['총점수(100)']) :
        print(normal_df['회사명'].iloc[i])


해성티피씨


In [143]:
print('최솟값, 최댓값, 평균값')
print(min(normal_df['E점수(100)']),max(normal_df['E점수(100)']), normal_df['E점수(100)'].mean())
print(min(normal_df['S점수(100)']),max(normal_df['S점수(100)']), normal_df['S점수(100)'].mean())
print(min(normal_df['G점수(100)']),max(normal_df['G점수(100)']), normal_df['G점수(100)'].mean())
print(min(normal_df['총점수(100)']),max(normal_df['총점수(100)']), normal_df['총점수(100)'].mean())

최솟값, 최댓값, 평균값
46.0 81.17977677844887 60.4843704783261
55.108709432048684 81.06027163061897 65.92117489176455
50.623999999999995 96.58562500943012 64.44034803456366
51.56703012814321 80.75717415277202 63.61529780155144


In [144]:
print('최솟값, 최댓값, 평균값')
print(min(normal_df['E점수(100)'])-max(normal_df['E점수(100)']), normal_df['E점수(100)'].mean())
print(min(normal_df['S점수(100)'])-max(normal_df['S점수(100)']), normal_df['S점수(100)'].mean())
print(min(normal_df['G점수(100)'])-max(normal_df['G점수(100)']), normal_df['G점수(100)'].mean())
print(min(normal_df['총점수(100)'])-max(normal_df['총점수(100)']), normal_df['총점수(100)'].mean())

최솟값, 최댓값, 평균값
-35.17977677844887 60.4843704783261
-25.951562198570286 65.92117489176455
-45.96162500943012 64.44034803456366
-29.190144024628815 63.61529780155144


In [131]:
normal_df[(normal_df['회사명'] == '해성티피씨')]

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
11094,해성티피씨,2013,20.0,18.0,45.000000,40.000000,40.000000,0.0000,0.0000,20.952381,20.036236,30.426667,85.952381,78.036236,70.426667,78.138428,0.000000,0.000000,0.000000,0.000000
11095,해성티피씨,2019,20.0,18.0,50.129142,46.002283,46.311955,0.7594,0.2406,20.000000,20.036236,30.426667,90.129142,84.038520,76.738622,83.635428,12.008498,19.201429,14.279251,15.907509
11096,해성티피씨,2020,20.0,18.0,49.243919,45.229248,45.209350,0.6263,0.3737,20.000000,20.036236,30.426667,89.243919,83.265484,75.636017,82.715140,-3.223091,-2.472956,-2.494373,-2.595114
11097,해성티피씨,2021,20.0,18.0,49.111116,44.925435,44.958215,0.6037,0.3963,20.000000,20.036236,30.426667,89.111116,82.961672,75.384882,82.485890,-0.483533,-0.971903,-0.568132,-0.716951
11098,해성티피씨,2022,20.0,18.0,48.888065,44.781957,44.694422,0.5723,0.4277,20.952381,20.036236,30.426667,89.840446,82.818193,75.121089,82.593243,5.854539,-0.458991,-0.596766,0.426040
11099,해성티피씨,2023,20.0,18.0,49.076566,44.865995,44.974124,0.5994,0.4006,20.952381,20.036236,30.426667,90.028947,82.902231,75.400790,82.777323,0.686330,0.268839,0.632756,0.495225
11100,해성티피씨,2024,20.0,18.0,48.379349,43.972501,43.973957,0.4918,0.5082,20.000000,20.036236,30.426667,88.379349,82.008737,74.400624,81.596236,-9.205229,-2.858306,-2.262632,-3.542291


In [96]:
normal_df

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,20.0,18.0,48.973870,44.838389,44.798021,0.5843,0.4157,20.0,20.625000,37.076,88.973870,83.463389,81.874021,24.846023,0.000000,0.000000,0.000000,0.000000
1,3S,2020,20.0,18.0,49.006956,44.884854,44.843184,0.5898,0.4102,20.0,20.625000,37.076,89.006956,83.509854,81.919184,24.969527,0.120467,0.148644,0.102172,0.123505
2,3S,2021,20.0,18.0,49.053542,44.903163,44.882442,0.5964,0.4036,20.0,20.625000,37.076,89.053542,83.528163,81.958442,25.058362,0.169619,0.058569,0.088810,0.088835
3,3S,2022,20.0,18.0,49.174962,45.051696,45.051856,0.6140,0.3860,20.0,20.625000,37.076,89.174962,83.676696,82.127856,25.487206,0.442091,0.475160,0.383259,0.428844
4,3S,2023,20.0,18.0,49.323874,45.193116,45.232982,0.6359,0.3641,20.0,20.625000,37.076,89.323874,83.818116,82.308982,25.933885,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,20.0,18.0,48.504595,44.304072,44.209404,0.5153,0.4847,20.0,20.687806,35.184,88.504595,82.991878,79.393404,20.567875,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,20.0,18.0,48.884252,44.744321,44.668615,0.5710,0.4290,20.0,20.687806,35.184,88.884252,83.432127,79.852615,21.806053,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,20.0,18.0,49.013553,44.849200,44.839852,0.5895,0.4106,20.0,20.687806,35.184,89.013553,83.537007,80.023852,22.185198,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,20.0,18.0,48.724020,44.507641,44.425581,0.5458,0.4542,20.0,20.687806,35.184,88.724020,83.195447,79.609581,21.168273,-1.054185,-1.092657,-0.937189,-1.016925


In [152]:
40000/   

6666.666666666667

In [145]:
# 결측치 0으로 채우기
normal_df['E점수(100) 차이'] = normal_df['E점수(100) 차이'].fillna(0)
normal_df['S점수(100) 차이'] = normal_df['S점수(100) 차이'].fillna(0)
normal_df['G점수(100) 차이'] = normal_df['G점수(100) 차이'].fillna(0)
normal_df['총점수(100) 차이'] = normal_df['총점수(100) 차이'].fillna(0)
normal_df

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,13.0,15.0,35.895478,39.676778,39.596042,0.5843,0.4157,13.0,11.875000,30.614,61.895478,66.551778,70.210042,66.219099,0.000000,0.000000,0.000000,0.000000
1,3S,2020,13.0,15.0,36.027823,39.769709,39.686369,0.5898,0.4102,13.0,11.875000,30.614,62.027823,66.644709,70.300369,66.324300,0.120467,0.148644,0.102172,0.123505
2,3S,2021,13.0,15.0,36.214167,39.806326,39.764884,0.5964,0.4036,13.0,11.875000,30.614,62.214167,66.681326,70.378884,66.424792,0.169619,0.058569,0.088810,0.088835
3,3S,2022,13.0,15.0,36.699849,40.103392,40.103712,0.6140,0.3860,13.0,11.875000,30.614,62.699849,66.978392,70.717712,66.798651,0.442091,0.475160,0.383259,0.428844
4,3S,2023,13.0,15.0,37.295495,40.386233,40.465963,0.6359,0.3641,13.0,11.875000,30.614,63.295495,67.261233,71.079963,67.212230,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,13.0,15.0,34.018381,38.608144,38.418807,0.5153,0.4847,13.0,12.063419,27.776,60.018381,65.671563,66.194807,63.961584,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,13.0,15.0,35.537006,39.488641,39.337229,0.5710,0.4290,13.0,12.063419,27.776,61.537006,66.552060,67.113229,65.067432,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,13.0,15.0,36.054211,39.698401,39.679705,0.5895,0.4106,13.0,12.063419,27.776,62.054211,66.761820,67.455705,65.423912,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,13.0,15.0,34.896082,39.015281,38.851161,0.5458,0.4542,13.0,12.063419,27.776,60.896082,66.078700,66.627161,64.533981,-1.054185,-1.092657,-0.937189,-1.016925


In [142]:
normal_df['총점수(100)'] = (normal_df['E점수(100)']+ normal_df['S점수(100)']+ normal_df['G점수(100)']) /3

In [146]:
normal_df

,회사명,년도,녹색점수,리콜점수,bert_e,bert_s,bert_g,Company_Pos,Company_Neg,특허점수,job_s,job_g,E점수(100),S점수(100),G점수(100),총점수(100),E점수(100) 차이,S점수(100) 차이,G점수(100) 차이,총점수(100) 차이
0,3S,2019,13.0,15.0,35.895478,39.676778,39.596042,0.5843,0.4157,13.0,11.875000,30.614,61.895478,66.551778,70.210042,66.219099,0.000000,0.000000,0.000000,0.000000
1,3S,2020,13.0,15.0,36.027823,39.769709,39.686369,0.5898,0.4102,13.0,11.875000,30.614,62.027823,66.644709,70.300369,66.324300,0.120467,0.148644,0.102172,0.123505
2,3S,2021,13.0,15.0,36.214167,39.806326,39.764884,0.5964,0.4036,13.0,11.875000,30.614,62.214167,66.681326,70.378884,66.424792,0.169619,0.058569,0.088810,0.088835
3,3S,2022,13.0,15.0,36.699849,40.103392,40.103712,0.6140,0.3860,13.0,11.875000,30.614,62.699849,66.978392,70.717712,66.798651,0.442091,0.475160,0.383259,0.428844
4,3S,2023,13.0,15.0,37.295495,40.386233,40.465963,0.6359,0.3641,13.0,11.875000,30.614,63.295495,67.261233,71.079963,67.212230,0.542186,0.452407,0.409752,0.446679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11374,힘스,2020,13.0,15.0,34.018381,38.608144,38.418807,0.5153,0.4847,13.0,12.063419,27.776,60.018381,65.671563,66.194807,63.961584,0.715609,1.064022,0.694795,0.845608
11375,힘스,2021,13.0,15.0,35.537006,39.488641,39.337229,0.5710,0.4290,13.0,12.063419,27.776,61.537006,66.552060,67.113229,65.067432,1.382326,1.408364,1.038853,1.238178
11376,힘스,2022,13.0,15.0,36.054211,39.698401,39.679705,0.5895,0.4106,13.0,12.063419,27.776,62.054211,66.761820,67.455705,65.423912,0.470784,0.335513,0.387383,0.379145
11377,힘스,2023,13.0,15.0,34.896082,39.015281,38.851161,0.5458,0.4542,13.0,12.063419,27.776,60.896082,66.078700,66.627161,64.533981,-1.054185,-1.092657,-0.937189,-1.016925


In [147]:
normal_df.isnull().sum()

회사명            0
년도             0
녹색점수           0
리콜점수           0
bert_e         0
bert_s         0
bert_g         0
Company_Pos    0
Company_Neg    0
특허점수           0
job_s          0
job_g          0
E점수(100)       0
S점수(100)       0
G점수(100)       0
총점수(100)       0
E점수(100) 차이    0
S점수(100) 차이    0
G점수(100) 차이    0
총점수(100) 차이    0
dtype: int64

In [149]:
normal_df.to_csv('총점수.csv', index=False)

db에 저장

In [151]:
company_detail = list(zip(normal_df['회사명'], normal_df['년도'], normal_df['Company_Pos'], normal_df['Company_Neg'], normal_df['E점수(100)'], normal_df['S점수(100)'], normal_df['G점수(100)'], normal_df['총점수(100)'], normal_df['E점수(100) 차이'], normal_df['S점수(100) 차이'], normal_df['G점수(100) 차이'], normal_df['총점수(100) 차이']))

db.insert_many(
    'tb_company_detail',
    'company, Year, Company_Pos, Company_Neg, E_Score, S_Score, G_Score, Total_Score, E_Score_Change, S_Score_Change, G_Score_Change, Total_Score_Change',
    company_detail
)

True

In [150]:
# 인덱스 초기화
db.truncate('tb_company_detail')

True